<a href="https://colab.research.google.com/github/gtbnhyujmj/Gold_Cross-Exchange-Arbitrage/blob/main/%5BV2%5D%20XAUT_BX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BX 查詢價格

In [1]:
import time
import requests
import hmac
from hashlib import sha256

In [2]:
# API 主網址
APIURL = "https://open-api.bingx.com"

In [3]:
# 你的 API 金鑰（可以先空著測試）
APIKEY = ""
SECRETKEY = ""

In [4]:
# 查詢 XAUT-USDT 的即時價格 (泰達公司出的泰達黃金)

def demo():
    # 請求資料可以是空的（GET 請求）
    payload = {}

    # API 的路徑
    path = '/openApi/swap/v1/ticker/price'

    # 請求方式
    method = "GET"

    # 要送出的參數
    paramsMap = {
        "symbol": "XAUT-USDT",  # 想查詢的幣種
        "timestamp": "1702718923479"  # 固定的測試值，後面會自動更新
    }

    # 將參數轉成 URL 格式字串
    paramsStr = parseParam(paramsMap)

    # 送出請求
    return send_request(method, path, paramsStr, payload)

In [5]:
# 對參數進行簽名（加密）

def get_sign(api_secret, payload):
    # 使用 SHA256 加密並回傳簽名值
    key_bytes = api_secret.encode("utf-8")
    payload_bytes = payload.encode("utf-8")
    signature = hmac.new(key_bytes, payload_bytes, digestmod=sha256).hexdigest()
    print("簽名結果 =", signature)
    return signature

In [6]:
# 發送 GET 請求

def send_request(method, path, urlpa, payload):
    # 手動組合 URL 字串（不使用 f-string）
    base_url = APIURL + path + "?" + urlpa + "&signature=" + get_sign(SECRETKEY, urlpa)
    print("請求網址 =", base_url)

    # 設定 headers，帶上 API 金鑰
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 送出請求
    response = requests.request(method, base_url, headers=headers, data=payload)

    # 回傳伺服器的回應
    return response.text

In [7]:
# 將參數轉為 URL 字串，並自動加入目前時間戳

def parseParam(paramsMap):
    # 將參數依照 key 排序
    sortedKeys = sorted(paramsMap)

    # 將每組 key=value 用 & 串接起來
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr = paramsStr + "&" + key + "=" + str(paramsMap[key])
        else:
            paramsStr = key + "=" + str(paramsMap[key])

    # 加上目前時間戳
    now_timestamp = "timestamp=" + str(int(time.time() * 1000))

    # 如果已經有參數，就補上 &timestamp
    if paramsStr != "":
        return paramsStr + "&" + now_timestamp
    else:
        return now_timestamp

In [8]:
# 主程式：呼叫 demo 函式
if __name__ == '__main__':
    print("執行結果:", demo())

簽名結果 = eeb690d11068438bfa1b416d6c1eca6cc12e52c84fdcf2347f218d06fd96d9fe
請求網址 = https://open-api.bingx.com/openApi/swap/v1/ticker/price?symbol=XAUT-USDT&timestamp=1702718923479&timestamp=1749950476523&signature=eeb690d11068438bfa1b416d6c1eca6cc12e52c84fdcf2347f218d06fd96d9fe
執行結果: {"code":0,"msg":"","data":{"symbol":"XAUT-USDT","price":"3471.5","time":1749950476607}}


In [9]:
# 自動排版
import json  # 加在檔案開頭

BX = demo()

# 在 print(result) 改成這樣：
parsed = json.loads(BX)
print(json.dumps(parsed, indent=2))

簽名結果 = 1608c6e80434e815ab271c1ff9defe172abbd0e6db5f848e9656c0f502657581
請求網址 = https://open-api.bingx.com/openApi/swap/v1/ticker/price?symbol=XAUT-USDT&timestamp=1702718923479&timestamp=1749950476832&signature=1608c6e80434e815ab271c1ff9defe172abbd0e6db5f848e9656c0f502657581
{
  "code": 0,
  "msg": "",
  "data": {
    "symbol": "XAUT-USDT",
    "price": "3471.5",
    "time": 1749950476807
  }
}


In [10]:
bingx_gold_price = parsed["data"]["price"]

bingx_gold_price

'3471.5'

# 交易所2 查詢價格

In [11]:
exchange_b_gold_price = parsed["data"]["price"]

exchange_b_gold_price

'3471.5'

# BX 開倉定義

In [12]:
import time
import requests
import hmac
from hashlib import sha256

In [13]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [14]:
def BX_place_order(symbol, side, positionSide, quantity, take_profit_price=None):

    """
    向 BingX 發送市價單下單指令，可選擇停利參數。

    symbol: 幣種（例如 "BTC-USDT"）
    side: "BUY" 或 "SELL"
    positionSide: "LONG" 或 "SHORT"
    quantity: 下單數量
    take_profit_price: 停利價格（選填）

    回傳值為 BingX API 的回應文字。
    """

    # API 資訊
    path = '/openApi/swap/v2/trade/order'
    method = "POST"

    # 組裝參數
    paramsMap = {
        "symbol": symbol,
        "side": side,
        "positionSide": positionSide,
        "type": "MARKET",  # 市價單
        "quantity": quantity
    }

    # 如有設定停利價格，就加入 takeProfit 條件
    if take_profit_price is not None:
        take_profit_dict = {
            "type": "TAKE_PROFIT_MARKET",
            "stopPrice": take_profit_price,
            "price": take_profit_price,
            "workingType": "MARK_PRICE"
        }
        import json
        paramsMap["takeProfit"] = json.dumps(take_profit_dict)

    # 加入時間戳並排序
    sortedKeys = sorted(paramsMap)
    paramsStr = "&".join(["%s=%s" % (k, paramsMap[k]) for k in sortedKeys])
    timestamp = str(int(time.time() * 1000))
    if paramsStr != "":
        paramsStr += "&timestamp=" + timestamp
    else:
        paramsStr = "timestamp=" + timestamp

    # 產生簽名
    sign_payload = paramsStr
    signature = hmac.new(SECRETKEY.encode("utf-8"), sign_payload.encode("utf-8"), digestmod=sha256).hexdigest()

    # 發送請求
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    response = requests.request(method, url, headers=headers, data={})
    return response.text

In [15]:
# 下單 BTC-USDT，多單，數量 5，不設停利
# result = BX_place_order("BTC-USDT", "BUY", "LONG", 5)

# 下單 ETH-USDT，空單，數量 2，停利設在 3500
# result = BX_place_order("ETH-USDT", "SELL", "SHORT", 2, take_profit_price=3500)

# print("回傳結果:", result)

In [16]:
# 因為是兩個或是未來多個交易所套利，所以應該是不用設定停利條件。

# BINGX關倉需要合約號，所以還要再寫個查合約號，並且記錄合約號。

# BX 合約號紀錄

In [17]:
import time
import requests
import hmac
from hashlib import sha256

In [18]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [19]:
def get_bingx_position_id(symbol):

    """
    查詢指定幣種持倉資訊，回傳該倉位的 positionId 字串。
    symbol: 幣種對（例如 "BNB-USDT"）

    回傳: positionId（字串），若查無則回傳 None。
    """

    # API 資訊
    path = '/openApi/swap/v2/user/positions'
    method = "GET"

    # 組裝請求參數
    paramsMap = {
        "symbol": symbol
    }

    # 排序參數並加入 timestamp
    sortedKeys = sorted(paramsMap)
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr += "&"
        paramsStr += key + "=" + str(paramsMap[key])
    paramsStr += "&timestamp=" + str(int(time.time() * 1000))

    # 簽名
    signature = hmac.new(
        SECRETKEY.encode("utf-8"),
        paramsStr.encode("utf-8"),
        digestmod=sha256
    ).hexdigest()

    # URL 組合
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    print("查詢請求：", url)

    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    # 發送請求
    response = requests.request(method, url, headers=headers, data={})
    try:
        result = response.json()
        position_id = result["data"]["positionId"]
        print("🎯 查到 positionId =", position_id)
        return str(position_id)
    except Exception as e:
        print("⚠️ 解析失敗或查無倉位：", e)
        return None

In [20]:
position_id = get_bingx_position_id("XAUT-USDT")

ID = []

if position_id is not None:
  ID.append(position_id)
  print("✅ 倉位 ID 已獲得：", position_id)
  print(ID)
else:
  print("❌ 查無倉位")

查詢請求： https://open-api.bingx.com/openApi/swap/v2/user/positions?symbol=XAUT-USDT&timestamp=1749950477274&signature=b6522c93edafefa526882885b8a0cdba65ad2d8638f55e5f10466ebf4979a974
⚠️ 解析失敗或查無倉位： 'data'
❌ 查無倉位


# 交易所2 開倉定義

# BX 關倉定義

In [21]:
import time
import requests
import hmac
from hashlib import sha256

In [22]:
APIURL = "https://open-api.bingx.com"
APIKEY = ""      # ✅ 請填入你的 API KEY
SECRETKEY = ""   # ✅ 請填入你的 SECRET KEY

In [23]:
def close_bingx_position(order_id, symbol):

    """
    取消 BingX 的合約訂單（平倉用）

    order_id: 要取消的訂單 ID
    symbol: 幣種對，如 "RNDR-USDT"

    回傳值為 API 的回應文字。
    """

    # API 路徑與方法
    path = '/openApi/swap/v2/trade/order'
    method = "DELETE"

    # 組合參數（不含簽名）
    paramsMap = {
        "orderId": order_id,
        "symbol": symbol
    }

    # 加入 timestamp 並轉為參數字串
    sortedKeys = sorted(paramsMap)
    paramsStr = ""
    for key in sortedKeys:
        if paramsStr != "":
            paramsStr = paramsStr + "&" + key + "=" + str(paramsMap[key])
        else:
            paramsStr = key + "=" + str(paramsMap[key])

    # 加入當前時間戳
    timestamp = str(int(time.time() * 1000))
    if paramsStr != "":
        paramsStr += "&timestamp=" + timestamp
    else:
        paramsStr = "timestamp=" + timestamp

    # 簽名處理
    signature = hmac.new(
        SECRETKEY.encode("utf-8"),
        paramsStr.encode("utf-8"),
        digestmod=sha256
    ).hexdigest()

    # 組合 URL
    url = APIURL + path + "?" + paramsStr + "&signature=" + signature
    print("取消訂單請求：", url)

    # 設定標頭與送出請求
    headers = {
        'X-BX-APIKEY': APIKEY,
    }

    response = requests.request(method, url, headers=headers, data={})
    return response.text

In [24]:
# 用 for in ID = [] 的模式關倉。

# 交易所2 關倉定義

# 比價

In [25]:
# GPT輸入:
# 這邊有不少問題，概念上是這樣，先把BX-B的價格算出來，這樣才可以確定是BX的黃金價格在高位，還是B的價格在相對高位。

# 對相對高位的那個交易所下空單，相反的下多單，不斷的監視價格。
# 當價格差距縮小的時候(這個時候兩邊都會獲利)，每間隔指定的點位，加一個單，直到BX-X為零，或是逼近零，這個時候無獲利空間，發出指令，兩邊所有倉位都平倉。

In [26]:
import time

In [27]:
CHECK_INTERVAL = 0.2  # 秒
ENTRY_SPREAD = 5      # 初始套利進場門檻（價差）
ADD_SPREAD_STEP = 1   # 每縮小幾 USDT 加一次單
CLOSE_SPREAD = 0.5    # 接近0視為無套利空間，平倉離場

In [28]:
# 初始進場狀態
entered = False
entry_spread = None
last_spread = None
add_count = 0

In [29]:
# 說明
# entered: 控制進場與否
# entry_spread: 記錄初始進場價差
# last_spread: 用來判斷是否縮小了 ADD_SPREAD_STEP 再加碼
# add_count: 記錄加倉次數
# spread > 0 代表 BingX 高於 B → 對 BX 空、對 B 多

In [ ]:
while True:
    bx = # get_bingx_price()
    b = # get_exchange_b_price()

    if bx is None or b is None:
        continue

    spread = bx - b
    print(f"BingX: {bx} | B: {b} | 差價: {spread:.2f}")

    if not entered:
        if abs(spread) >= ENTRY_SPREAD:

            # 初次進場判斷：誰高誰空
            if spread > 0:
                # BX 在高點，對 BX 空，對 B 多
                open_short("BingX", bx)  #result = BX_place_order("ETH-USDT", "SELL", "SHORT", 2, take_profit_price=3500)
                open_long("ExchangeB", b) #result = BX_place_order("BTC-USDT", "BUY", "LONG", 5)
            else:
                open_long("BingX", bx)
                open_short("ExchangeB", b)

            entered = True
            entry_spread = abs(spread)
            last_spread = abs(spread)
    else:
        # 已進場後監控
        current_spread = abs(spread)

        if current_spread < last_spread - ADD_SPREAD_STEP:
            print(f"🔁 價差縮小到 {current_spread:.2f}，加碼第 {add_count + 1} 次")
            if spread > 0:
                open_short("BingX", bx)
                open_long("ExchangeB", b)
            else:
                open_long("BingX", bx)
                open_short("ExchangeB", b)

            last_spread = current_spread
            add_count += 1

        elif current_spread <= CLOSE_SPREAD:
            print("🏁 價差逼近 0，執行全部平倉")
            close_BX_positions() # BX 應該是用 for in ID = [] 的模式，用上面的close_bingx_position關倉。
            # clode_B_positions()
            break

    time.sleep(CHECK_INTERVAL)

In [ ]:
# 2025/06/15